In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content/drive/My Drive/PROJECT - NLP/semantic-segmentation/data/text

/content/drive/My Drive/PROJECT - NLP/semantic-segmentation/data/text


In [0]:
!pip install transformers

     |████████████████████████████████| 573kB 3.5MB/s 
     |████████████████████████████████| 3.7MB 16.1MB/s 
     |████████████████████████████████| 1.0MB 42.1MB/s 
     |████████████████████████████████| 890kB 48.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=4b6e909ac45851e743c671bcca532b690c019e279efb1cc4522ab258f3e844e5
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [0]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [0]:
from transformers import BertModel, AdamW, BertConfig

model = BertModel.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.  
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [0]:
%cd /content/drive/My Drive/PROJECT - NLP/semantic-segmentation

/content/drive/My Drive/PROJECT - NLP/semantic-segmentation


In [0]:
Documnets = []
with open('categories.txt','r+') as f:
    doc = f.read()
    for fil in doc.split('\n'):
      if len(fil.split('\t')) == 2:
        for names in fil.split('\t')[1].split(" "):
          Documnets.append(names)

In [0]:
import os
import io
import re
import pandas as pd
import numpy as np

In [0]:
path = '/content/drive/My Drive/PROJECT - NLP/semantic-segmentation/data/text/files'

In [0]:
Class = []
file1 = []

In [0]:
for filename in Documnets:
  filename = filename+'.txt'
  with io.open(os.path.join(path,filename),'r') as f:
    file = f.read()
  emd = []
  for i in range(len(file.split('\n'))):
    if len(file.split('\n')[i].split('\t')) == 2:
      s = file.split('\n')[i].split('\t')
      encoded_dict = tokenizer.encode_plus(s[0],add_special_tokens = True, max_length = 64,pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        )
      input_ids = encoded_dict['input_ids']
      attention_masks = encoded_dict['attention_mask']
      t = model(input_ids.cuda(),attention_mask = attention_masks.cuda())[0]
      #print(np.mean(t.detach().cpu().numpy().reshape(-1,768),axis = 0).reshape(1,-1))
      emd.append(np.mean(t.detach().cpu().numpy().reshape(-1,768),axis = 0).reshape(1,-1))
      Class.append(s[1])
      file1.append(filename)
  emd = np.array(emd)
  emd = emd.reshape(emd.shape[0],-1)
  print('Saving to '+filename)
  fi = open(os.path.join(path+'/pretrained-embeddings/',filename), "a")
  np.savetxt(os.path.join(path,'pretrained-embeddings/',filename),emd,delimiter=',',newline='\n')
  #fi.write(s[1]+'\n')
  fi.close()

Saving to 1953_L_1.txt
Saving to 1973_S_68.txt
Saving to 1978_M_13.txt
Saving to 1994_M_69.txt
Saving to 1996_B_72.txt
Saving to 2008_S_549.txt
Saving to 2008_S_1411.txt
Saving to 2011_S_308.txt
Saving to 2014_J_33.txt
Saving to 2015_S_368.txt
Saving to 1953_S_23.txt
Saving to 2007_B_76.txt
Saving to 1995_S_317.txt
Saving to 2000_C_151.txt
Saving to 1976_T_9.txt
Saving to 1987_M_123.txt
Saving to 2007_S_632.txt
Saving to 2004_C_129.txt
Saving to 2008_C_166.txt
Saving to 1971_S_1.txt
Saving to 2005_S_388.txt
Saving to 2006_A_36.txt
Saving to 2008_I_54.txt
Saving to 2010_S_431.txt
Saving to 2011_I_16.txt
Saving to 2012_S_270.txt
Saving to 2014_R_41.txt
Saving to 1963_S_59.txt
Saving to 1987_C_108.txt
Saving to 2001_S_1131.txt
Saving to 2006_A_136.txt
Saving to 2007_S_608.txt
Saving to 2007_U_18.txt
Saving to 2009_S_146.txt
Saving to 2015_J_10.txt
Saving to 1996_T_169.txt
Saving to 2007_C_121.txt
Saving to 2008_P_8.txt
Saving to 2009_B_16.txt
Saving to 2010_J_55.txt
Saving to 2001_A_234.t